Series

In [ ]:
import pandas as pd

data = [100, 102, 104, 200, 202]

series = pd.Series(data, index=["a", "b", "c", "d", "e"])

series[series >= 200] #prints values greater than or equal to 200


d    200
e    202
dtype: int64

In [ ]:
import pandas as pd

calories = {"Day 1": 1750, "Day 2": 2100, "Day 3": 1700}
series = pd.Series(calories)

#two ways to update the series
series["Day 3"] = 1800
series.loc["Day 3"] = 300

#eaten more than 2000
series[series > 2000]



Day 2    2100
dtype: int64

DataFrames

In [121]:
data = {"name": ["spongebob", "patric", "squidward"],
        "age": [30,35,50]
}

df = pd.DataFrame(data, index=[f"Employee {i}" for i in range(1,4)]) #df
df.loc["Employee 2"] #look at employee 2
df["Adress"] = ["alabama", "new mexico", "newkurky"] # new column

#new row
new_rows = pd.DataFrame([{"name": "sandy", "age": 30, "Adress": "pinaple"},
                        {"name": "crabbyboy", "age": 45, "Adress": "in the cage"}],
                        index=["Employee 4","masterboy"])
df = pd.concat([df, new_rows]) #concatination adding the two dataframes
df

,name,age,Adress
Employee 1,spongebob,30,alabama
Employee 2,patric,35,new mexico
Employee 3,squidward,50,newkurky
Employee 4,sandy,30,pinaple
masterboy,crabbyboy,45,in the cage


Importing

In [124]:
df = pd.read_csv("pokemon.csv", sep=",", index_col="No")

#selection by column
df[["Name","Height","Weight"]] #prints a df with columns name, height and weight

#make a copy dataframe
new_df = df[["Name","Height","Weight"]].copy()
new_df["Height"] = new_df["Height"]/10

df.query("Name == 'Pikachu'")[["Name","Height","Legendary"]] #to find a spesific pokemon, use singel quotas

#set name as index to show items inbetween
df = df.set_index("Name")
df.loc["Charizard":"Blastoise"]

,Type1,Type2,Height,Weight,Legendary
Name,,,,,
Charizard,Fire,Flying,1.7,90.5,0
Squirtle,Water,NaN,0.5,9.0,0
Wartortle,Water,NaN,1.0,22.5,0
Blastoise,Water,NaN,1.6,85.5,0


In [125]:

#search for a pokemon of choice
prompt = input("What pokemon do you want?: ")
try:
    print(df.loc[prompt])
except KeyError:
    print(f"pokemon {prompt} not found")


Type1          Fire
Type2        Flying
Height          1.7
Weight         90.5
Legendary         0
Name: Charizard, dtype: object


Filtering

In [144]:
#keeping the rows mathcing a condition
#df = df.query("Weight > 10") #dropping rows with lower weigth
tall_pokemon_df = df[df["Height"]>2]
heavy_pokemon_df = df[df["Weight"]> 100]
ledgendary_pokemon_df = df[df["Legendary"]== True] #True == 1, could also write ==1
df_extended = pd.concat([tall_pokemon_df, heavy_pokemon_df, ledgendary_pokemon_df]) #making a df with the new dataframes
water_pokemon = df[(df["Type1"]=="Water") | #| == or (it means or)
                   (df["Type2"]=="Water")
                   ]
ff_pokemon = df[(df["Type1"]== "Fire") & 
                (df["Type2"]=="Flying")
                ]
ff_pokemon


,Type1,Type2,Height,Weight,Legendary
Name,,,,,
Charizard,Fire,Flying,1.7,90.5,0
Moltres,Fire,Flying,2.0,60.0,1


Aggregation

In [156]:
#reduce a set of values into a single value. summerize and analyze data.
#often used with the groupby() function
#df.groupby(["Height"]).mean()
df.mean(numeric_only=True) #finding the means of numeric values
df.sum(numeric_only=True) #summing all the numbers
df.median(numeric_only=True) #median values
df.max(numeric_only=True) #max values
df.query("Height == '8.8'")
df[df["Height"]>8.7]



,Type1,Type2,Height,Weight,Legendary
Name,,,,,
Onix,Rock,Ground,8.8,210.0,0
